In [11]:
# Necessary library imports
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
import kmedoids
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import shapefile
import geocoder as geo
import haversine

%matplotlib inline

In [12]:
# Importing the data from file
parking_counts = pd.read_csv('data/Parking_Counts.csv')
restaurant_counts = pd.read_csv('data/Restaurant_Counts.csv')
stop_counts = pd.read_csv('data/Stop_Counts.csv')

In [6]:
# Calculating values for the grid
long_max = stop_counts['longitude'].max()
long_min = stop_counts['longitude'].min()
lat_max = stop_counts['latitude'].max()
lat_min = stop_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    stop_counts.loc[stop_counts['stop_id'] == s,'row'] =  (stop_counts.loc[stop_counts['stop_id'] == s,'latitude'] - lat_min) // lat_gap
    stop_counts.loc[stop_counts['stop_id'] == s,'col'] = (stop_counts.loc[stop_counts['stop_id'] == s,'longitude'] - long_min) // long_gap

stop_counts['stop_id'].apply(classify_blocks)

stop_counts

,Unnamed: 0,total_count,stop_id,latitude,longitude,am_early,am_peak,midday,pm_peak,pm_late,row,col
0,0,214,75000,32.54,-117.03,48.0,36.0,40.0,70.0,20.0,-0.0,7.0
1,1,214,75000,32.54,-117.03,48.0,36.0,40.0,70.0,20.0,-0.0,7.0
2,2,101,75002,32.56,-117.05,21.0,18.0,20.0,32.0,10.0,0.0,6.0
3,3,98,75003,32.56,-117.05,20.0,16.0,20.0,32.0,10.0,0.0,6.0
4,4,101,75004,32.57,-117.07,21.0,18.0,20.0,32.0,10.0,1.0,5.0
5,5,101,75005,32.57,-117.07,23.0,17.0,19.0,32.0,10.0,1.0,5.0
6,6,101,75006,32.59,-117.08,20.0,19.0,20.0,32.0,10.0,1.0,5.0
7,7,95,75007,32.59,-117.08,16.0,17.0,20.0,32.0,10.0,1.0,5.0
8,8,101,75008,32.60,-117.08,20.0,19.0,20.0,31.0,11.0,2.0,5.0
9,9,99,75009,32.60,-117.08,21.0,16.0,20.0,32.0,10.0,2.0,5.0


In [8]:
restaurant_counts.rename(columns={'lat':'latitude'}, inplace=True)
restaurant_counts.rename(columns={'lon':'longitude'}, inplace=True)
restaurant_counts.rename(columns={'AM early':'am_early'}, inplace=True)
restaurant_counts.rename(columns={'AM peak':'am_peak'}, inplace=True)
restaurant_counts.rename(columns={'Mid-day':'midday'}, inplace=True)
restaurant_counts.rename(columns={'PM peak':'pm_peak'}, inplace=True)
restaurant_counts.rename(columns={'PM late':'pm_late'}, inplace=True)
restaurant_counts.rename(columns={'Total Score':'total_count'}, inplace=True)
restaurant_counts.rename(columns={'Type of Place':'type'}, inplace=True)
restaurant_counts.rename(columns={'Unnamed: 0':'id'}, inplace=True)
parking_counts.rename(columns={'pole_id':'id'}, inplace=True)
stop_counts.rename(columns={'stop_id':'id'}, inplace=True)
parking_counts['type'] = 'parking'
stop_counts['type'] = 'transit'
location_counts = restaurant_counts.append(parking_counts)
# Add to analysis once latlong pairs are fixed
#location_counts = location_counts.append(stop_counts)
#location_counts['ix'] 
#stop_counts

In [10]:
long_max = location_counts['longitude'].max()
long_min = location_counts['longitude'].min()
lat_max = location_counts['latitude'].max()
lat_min = location_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    location_counts.loc[location_counts['id'] == s,'row'] =  (location_counts.loc[location_counts['id'] == s,'latitude'] - lat_min) // lat_gap
    location_counts.loc[location_counts['id'] == s,'col'] =  (location_counts.loc[location_counts['id'] == s,'longitude'] - long_min) // long_gap

location_counts['id'].apply(classify_blocks)

location_counts.loc[location_counts['row'] == -0.0, 'row'] = 10.0
location_counts.loc[location_counts['col'] == -0.0, 'col'] = 10.0
location_counts.to_csv('Label_issue.csv')

In [ ]:
# Wrangling locations for creating grid
latlongs = location[['latitude','longitude']]
latlongs = latlongs.dropna()
#restaurant_loc = restaurant[['lat','lon']].as_matrix()
#latlongs

In [ ]:
max_lat = max(latlongs['latitude'])
min_lat = min(latlongs['latitude'])
max_lon = max(latlongs['longitude'])
min_lon = min(latlongs['longitude'])

In [ ]:
def create_grid(min_lat,min_lon,max_lat,max_lon,size):
    grid = np.array([])
    grid_lon = max_lon - min_lon
    grid_lat = max_lat - min_lat
    grid_lon_size = grid_lon/size
    grid_lat_size = grid_lat/size
    i = 0
    j = 0
    for i in range(size):
        for j in range(size):
            grid[i, j] = (min_lat + i*grid_lat_size, min_lon + j*grid_lon_size)
    return grid
sd_grid = create_grid(min_lat,min_lon,max_lat,max_lon,10)
sd_grid

In [ ]:
# Setting the label index for each of the dataframes
#restaurant_info = restaurant
#restaurant_info['label'] = 0
label_loc = location
label_loc['label'] = 0
label_loc

In [ ]:
# Defining function that runs k-mediods algorithm
def center_label(locations, location_info, num_centers):
    
    dist = metrics.pairwise_distances(locations, metric= 'manhattan')
    M, C = kmedoids.kMedoids(dist, num_centers)

    #print('medoids:')
    #for point_idx in M:
        #print( locations[point_idx][0] )
        #print("1:") print( locations[point_idx][1] )

    #print('clustering result:')
    for label in C:
        for point_idx in C[label]:
            lat = locations[point_idx][0]
            lon = locations[point_idx][1]
            location_info.loc[(location_info['latitude'] == lat) & (location_info['longitude'] == lon),'label'] = label
           
    return M, C

In [ ]:
# Runs the k-medoid algorithm to get centers and label locations
# RUN ONLY once per analysis per dataframe, running it out of order will give wrong labelling
centers, labeled_locations = center_label(latlongs, label_loc,15)

#label_loc.to_csv("Labeled-Locations.csv")

In [ ]:
total_count_loc = label_loc[['label','total_count','latitude','longitude']]
total_count_loc.to_csv("Total-Count-Labeled-Locations.csv")

In [ ]:
am_early_loc = label_loc[['label','am_early','latitude','longitude']]
am_early_loc.to_csv('AM-Early-Labeled-Locations.csv')

In [ ]:
am_peak_loc = label_loc[['label','am_peak','latitude','longitude']]
am_peak_loc.to_csv('AM-Peak-Labeled-Locations.csv')

In [ ]:
pm_peak_loc = label_loc[['label','pm_peak','latitude','longitude']]
pm_peak_loc.to_csv('PM-Peak-Labeled-Locations.csv')

In [ ]:
pm_late_loc = label_loc[['label','pm_late','latitude','longitude']]
pm_late_loc.to_csv('PM-Late-Labeled-Locations.csv')

In [ ]:
midday_loc = label_loc[['label','midday','latitude','longitude']]
midday_loc.to_csv('Midday-Labeled-Locations.csv')

In [ ]:
# Gets the values and ranking for AM_early walkscore
am_early_values = label_loc[['label','am_early']]
am_early_values = am_early_values.groupby(['label']).sum().reset_index()
am_early_values['lat'] = 0
am_early_values['lon'] = 0
for i in range(len(am_early_values)):
    center_index = centers[am_early_values['label'][i]]
    latlon = transport_loc[center_index]
    am_early_values.ix[i, 'lat'] = latlon[0]
    am_early_values.ix[i, 'lon'] = latlon[1]
    info = transport_info.ix[center_index]
    
am_early_values

In [ ]:
am_peak_values = transport_info[['label','am_peak']]
am_peak = am_peak_values.groupby(['label']).sum().reset_index().sort_values('am_peak',ascending=False).reset_index()
print am_peak

In [ ]:
midday_values = transport_info[['label','midday']]
midday = midday_values.groupby(['label']).sum().reset_index().sort_values('midday',ascending=False).reset_index()
print midday

In [ ]:
pm_peak_values = transport_info[['label','pm_peak']]
pm_peak = pm_peak_values.groupby(['label']).sum().reset_index().sort_values('pm_peak',ascending=False).reset_index()
print pm_peak

In [ ]:
pm_late_values = transport_info[['label','pm_late']]
pm_late = pm_late_values.groupby(['label']).sum().reset_index().sort_values('pm_late',ascending=False).reset_index()
print pm_late

In [ ]:
print pm_late['label']